<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [2]:
# avoid decoding problems
df = pd.read_csv("train.csv")
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [18]:
df.shape

(404290, 6)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer()
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [7]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

In [10]:
from multiprocessing import Pool

In [11]:
import vect

In [12]:
pool = Pool(8)

In [13]:
vecs1 = pool.map(vect.vectorize, list(df['question1']))

In [20]:
print(len(vecs1[0]))
df['q1_feats_m'] = vecs1

96


In [21]:
vecs2 = pool.map(vect.vectorize, list(df['question2']))
df['q2_feats_m'] = vecs2

In [22]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [23]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [24]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,63,63,43,47,0.166667
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,28,24,9,14,0.039216
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,35,56,0.175000


In [25]:
# data before preprocessing 
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [26]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,75.956332,83.516817,75.882494,-58.858828,32.925084,23.283052,-31.806333,42.283992,-83.365978,55.004482,...,4.956905,83.255654,39.840175,-23.889550,3.266861,-49.650501,-107.540745,7.224697,64.357253,-6.908301
1,8.713562,-9.401626,2.696431,-17.054784,16.019740,-17.218485,-15.524917,-0.666317,-28.302419,25.389761,...,14.272849,2.756905,1.841273,-5.902120,-12.178115,-8.092485,-23.226474,4.462254,26.484037,-4.724977
2,29.620665,63.508035,24.289555,-49.179939,60.521525,-32.358392,-14.911530,12.659766,-126.476235,87.435216,...,81.116922,10.360326,-17.105197,32.520174,-25.506022,-88.022750,-136.817587,-55.106682,2.936054,-9.213937
3,23.183091,-48.785755,52.635587,-55.273645,12.299671,-10.207744,-27.799183,117.584649,28.038974,25.932145,...,-41.082597,28.920714,-84.750659,-34.576196,28.052577,-21.162300,41.440663,-31.039089,-90.496786,7.101299
4,122.719936,193.588517,132.864780,-103.585525,57.025020,-73.228229,-37.825657,50.490053,-55.909131,255.700764,...,-37.348823,64.666783,-3.493104,-69.870417,33.614204,-82.194303,-135.570828,-4.058554,5.328930,-2.407595


In [27]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,71.166095,61.084169,67.461856,-61.909868,28.595977,16.333390,-42.691287,35.554835,-71.820780,62.636445,...,10.332961,89.823333,34.349990,-40.657773,16.502075,-43.736880,-95.930907,5.221644,32.724671,-9.706636
1,32.168524,63.262730,106.310031,-61.664102,-13.772635,-5.248583,108.831181,73.249085,-8.413203,21.529303,...,-2.780060,34.276505,-53.074817,-2.882145,-45.037438,-23.793390,-53.461048,-10.638702,-39.707417,-19.214516
2,-15.604672,-16.833485,-18.926706,-16.558891,6.655758,51.081671,35.392790,11.976254,-87.198730,36.477342,...,43.740419,33.507118,85.186742,79.271670,58.702182,11.045117,-24.040915,-15.654223,43.610502,23.809840
3,8.678897,-17.929347,4.942418,-1.468426,37.845104,-16.491879,-29.807733,5.957089,-41.271073,39.000347,...,14.101479,28.123343,14.460927,44.480100,-30.488636,63.497832,5.595095,-17.837029,5.876008,-9.416763
4,-8.833060,39.803641,57.952059,-35.447827,4.345990,11.481218,-40.318149,23.055140,-53.736109,47.791536,...,35.415687,28.584591,-9.907145,-21.254855,25.226502,-9.549214,-32.263763,-1.037475,-15.853993,-6.474173


In [28]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 96
Number of features in question2 w2v  dataframe : 96
Number of features in final dataframe  : 221


In [29]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('final_features.csv')